In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel import PromptTemplateConfig, PromptTemplate, SemanticFunctionConfig

In [39]:

load_dotenv()
OPENAI_ENDPOINT = os.getenv("OPENAI_USEAST3_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_USEAST3_API_KEY")



In [40]:
kernel = sk.Kernel()


gpt35 = AzureChatCompletion(deployment_name="gpt-35-turbo", # yours may be different
                            endpoint=OPENAI_ENDPOINT,
                            api_key=OPENAI_API_KEY)

gpt4 = AzureChatCompletion(deployment_name="gpt-4", # yours may be different
                            endpoint=OPENAI_ENDPOINT,
                            api_key=OPENAI_API_KEY)

kernel.add_chat_service("gpt35", gpt35)
kernel.add_chat_service("gpt4", gpt4)



In [41]:
prompt = """knock, knock? Who’s there? {{$input}}. {{$input}} who?"""

knock = kernel.create_semantic_function(prompt, temperature=0.8)

response = knock("Dishes")



In [42]:
print(response)

Dishes me, the dishes won't wash themselves!


In [43]:
jokes_plugin = kernel.import_semantic_skill_from_directory("plugins", "jokes")


In [44]:
response = jokes_plugin["knock_knock_joke"]("Dishes")
print(response)

Dishes the police, open up!


In [47]:
response = jokes_plugin["cross_the_road_joke"]("Atom")
print(response)

Because it wanted to split from its current path!
